In [110]:
import numpy as np
import pandas as pd
import sys, os
import collections
from collections import OrderedDict
import matplotlib.pyplot as plt
import Bio
import sklearn

### This section is for table reading:

In [3]:
os.chdir('../data')
root = os.getcwd()
data = root + '/affinity_data.csv'
peptide_file = root + '/binding_peptides.fas'
HLA_file = root + '/HLA_aln.fas'

def to_number(s):
    try:
        s1 = float(s)
        return s1
    except ValueError:
        return s

os.chdir('../src')

In [69]:
# Additional block for data manipulation

#data_ref = root+'/affinity_data_refined.csv'
#df = df.replace('-', 50000)
#res = df.applymap(lambda x: to_number(x))
#res['DPB1:04:01'][0]
#df.to_csv(data_ref,sep=',', mode='w', index=False)

### Data distribution check

In [4]:
df = pd.read_csv(data, index_col=0)
data_matrix = df.as_matrix()
df.stack().hist(color='k', alpha=0.5, bins=100)
plt.show()
Y_dist_tmp = [i for i in list(data_matrix.flatten().astype(int)) if i < 50000]
Y_dist = np.array(Y_dist_tmp)

In [5]:
import numpy as np 
import pylab 
import scipy.stats as stats
from scipy.stats import expon
from scipy.stats import poisson

#expon_example = np.random.exponential(scale=50000, size=17000)
#measurements = np.where(expon_example >= 50000)[0]

stats.probplot(Y_dist, dist="norm", plot=pylab)
#r = stats.poisson.rvs(1.5)
#r = expon.ppf([0.001, 0.5, 0.999])
#r = stats.poisson(2)
#r = stats.
stats.probplot(Y_dist, dist=r, plot=pylab)
pylab.show()

NameError: name 'r' is not defined

### Nested Dict HLA-peptide IC50: 

In [6]:
class NestedDict(dict):
    def __missing__(self, key):
        self[key] = type(self)()
        return self[key]

affinity = NestedDict()     
df = pd.read_csv(data, index_col=0)
for k,v in df.stack().iteritems():
    affinity[k[0]][k[1]] = v

### HQI function

In [7]:
from encode_HQI8 import *
from Bio import SeqIO
import itertools

'''
#Itertools chaining example: 
a = [[1,2,3],[4,5,6]]
list(itertools.chain(*a))
'''

def encode_seq(entry):
    tmp = encode_aaindex_features(entry)
    aaindex = list(itertools.chain(*tmp))    
    return aaindex

# This operation normalize the data 
def encode_seq_norm(entry, max_vals, min_vals):
    delta = max_vals - min_vals
    tmp = encode_aaindex_features(entry)
    vals = list(itertools.chain(*[list((t - min_vals)/delta) for t in tmp]))
    return vals
    
def min_max(entry):
    tmp = encode_aaindex_features(entry)
    max_vals = np.amax(tmp, axis=0)
    min_vals = np.amin(tmp, axis=0)
    return max_vals, min_vals

#res_1 = encode_seq('MASSSSVLLVVVLFA')
#res_2 = encode_seq_norm('MASSSSVLLVVVLFA', max_vals, min_vals)
#print res_1
#print res_2

### Featurizing function

In [8]:
def featurize(entries):
    max_vals, min_vals = min_max('GAVLIPFYWSTCMNQKRHDE')
    feature_dict = {}
    for e in entries[0:]:
        #feature_dict[e.id] = encode_seq(e.seq)                         # Not-normalized data
        feature_dict[e.id] = encode_seq_norm(e.seq, max_vals, min_vals) # Normalized data
    return feature_dict

### Peptide features: 

In [9]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

peptides = list(SeqIO.parse(peptide_file, 'fasta'))
pep_dict = featurize(peptides)
#pep_dict

rec_test = SeqRecord(Seq(''.join('Q'), IUPAC.protein), id='id')
#featurize([rec_test])

### HLA descriptors' features:

In [10]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC

HLAs = list(SeqIO.parse(HLA_file, 'fasta'))
HLA_desc = OrderedDict([
                        ('b9', 40),
                        ('b11', 42),
                        ('b13', 44),
                        ('b28', 59),
                        ('b30', 61),
                        ('b37', 68),
                        ('b47', 78),
                        ('b57', 88),
                        ('b60', 91),
                        #('b61', 92),
                        ('b67', 98),
                        ('b70', 101),
                        ('b71', 102),
                        ('b74', 105),
                        ('b78', 109),
                        #('b81', 112),
                        #('b82', 113),
                        ('b85', 116),
                    ])

def make_SeqRecord(HLAs, HLA_desc):
    HLA_list = []
    for h in HLAs:
        #print h.id, h.seq[HLA_descriptors['b9']], h.seq[HLA_descriptors['b11']], h.seq[HLA_descriptors['b13']]
        sele = [h.seq[i] for i in HLA_desc.values()]
        #print h.id, ''.join(sele)
        record = SeqRecord(Seq(''.join(sele),
                   IUPAC.protein),
                   id=h.id)
        HLA_list.append(record)
    return featurize(HLA_list)

    
HLAs_dict = make_SeqRecord(HLAs, HLA_desc)

### Combining HLA/peptide features with affinity value into new file:

In [10]:
fo = open('dataset_norm_v2.csv', 'a+')
features_len = len(HLA_desc) * len(HQI8_descriptors) + 15 * len(HQI8_descriptors)
features_count = ['f'+ str(i) for i in range(1,features_len+1)]
header = 'id,peptide,HLA,' + ','.join(str(x) for x in features_count) + ',IC50\n'
fo.write(header)

In [11]:
# Iterates over the elements within the 2-level dict
i=1
for k,v in affinity.iteritems():
    for k1,v1 in v.iteritems():
        features = list(itertools.chain(*[pep_dict[k], HLAs_dict[k1]]))
        #message = k + '\t' + k1 + '\t' + ','.join(str(x) for x in features) + '\t' + str(v1) + '\n'
        #message = k + '_' + k1 + ',' + ','.join(str(x) for x in features) + ',' + str(v1) + '\n'
        message = str(i) + ',' + k + ',' + k1 + ',' +','.join(str(x) for x in features) + ',' + str(v1) + '\n'
        fo.write(message)
        i += 1

### Data -> X, y

In [86]:
df = pd.read_csv('../dataset/dataset_norm_v2.csv', index_col=0)
df = df[df.IC50 < 50000]
df = df[df.IC50 > 0]
X_df = df.drop(['IC50','peptide','HLA'], axis=1)
y_df = df['IC50']

# We initially transform the data into 
X = X_df.as_matrix()
y = y_df.as_matrix()
target_names = X_df.columns.values

#X_df.loc[17172]
#len(X[2:122])
#print len(X), len(y)
#df_test.loc[1:30, ['peptide','HLA','f1']]

### PCA

In [283]:
print(__doc__)

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.lda import LDA

n_comp = 250
pca = PCA(n_components=n_comp)
X_r = pca.fit(X).transform(X)

# Percentage of variance explained for each components
#print('explained variance ratio (first %s components): \n %s') %(n_comp, str(pca.explained_variance_ratio_))
eigenv = pca.explained_variance_ratio_
plt.bar (range(len(eigenv)), eigenv)
plt.show()


#Itertools chaining example: 
a = [[1,2,3],[4,5,6]]
list(itertools.chain(*a))



### Feature selection 

In [12]:
import sklearn.feature_selection
f_select = sklearn.feature_selection.f_regression(X, y, center=True)

In [13]:
import operator
feat_zip = zip(f_select[1], list(target_names))

In [278]:
a = [1,4,3,2]
b = ['a','d','c','b']
[(x,y) for (y,x) in sorted(zip(a,b))]

[('a', 1), ('b', 2), ('c', 3), ('d', 4)]

In [70]:
import math

def feat_seed(num):
    res = math.floor((num - 0.1)/4) + 1
    return int(res)

test_dict = {}
feat_importance = [(x,y) for (y,x) in sorted(zip(f_select[1],list(target_names)))]
feats = [str(feat_seed(int(x[1:]))) for (y,x) in sorted(zip(f_select[1],list(target_names)))] # AA importance rank
points = list(reversed(range(1, len(feats)+1)))
feat_points = zip(feats, points)

feat_points
for i in feat_points:
    if test_dict[i[0]]:
        test_dict[i[0]] = test_dict[i[0]] + i[1]
    else:
        test_dict[i[0]] = i[1]

NameError: name 'f_select' is not defined

### Regression Tree Model 

In [114]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.cross_validation import cross_val_score


estimator = RandomForestRegressor(random_state=0, n_estimators=10)
estimator.fit(X,y)
estimator.score(X, y, sample_weight=None)

0.84411029386176006

In [102]:
from sklearn.metrics import r2_score

def score_fun(X, y, estimator):
    sample_weight = None
    return r2_score(y, estimator.predict(X), sample_weight=sample_weight)

score_fun(X, y, estimator)

0.89237208772342047

In [115]:
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn import datasets
from sklearn import cross_validation

rf = ensemble.RandomForestRegressor(max_features='auto')
X_test, y_test = datasets.make_regression(10000, 10)
#score = cross_validation.cross_val_score(rf, X_test, y_test)
score = cross_validation.cross_val_score(estimator, X, y)

print score


[ 0.03552822 -0.0358004  -0.02397521]


### Regression tree test

In [66]:
import numpy as np
from sklearn.preprocessing import LabelEncoder  
from sklearn.ensemble import RandomForestRegressor

X_test = np.asarray([('a',1,2),('b',2,3),('a',3,2),('c',1,3)]) 
y_test = np.asarray([1,2.5,3,4])

# transform 1st column to numbers
X_test[:, 0] = LabelEncoder().fit_transform(X_test[:,0]) 

regressor = RandomForestRegressor(n_estimators=150, min_samples_split=1)
regressor.fit(X_test, y_test)
print X_test
print y_test
print regressor.predict(X_test)

[['0' '1' '2']
 ['1' '2' '3']
 ['0' '3' '2']
 ['2' '1' '3']]
[ 1.   2.5  3.   4. ]
[ 1.84333333  2.54        2.56666667  3.11      ]
